# EWC

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 100
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/ewc.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'ewc_lambda': 50.0
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = EWC(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/50 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 110.18it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 111.43it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 103.90it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 107.68it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 107.01it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 118.62it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9102
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 118.15it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1245
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 119.63it/s]
>

  2%|▏         | 1/50 [00:59<48:16, 59.12s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.0892
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1295
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 96.40it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 98.27it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 98.76it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 97.03it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 97.71it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 118.39it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0127
	ExperienceForgetting/eval_phase/test_stream

  4%|▍         | 2/50 [02:00<48:22, 60.47s/it]

...


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1095
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.5411
	StreamForgetting/eval_phase/test_stream = 0.5411
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3541
-- >> Start of training phase << --
100%|██████████| 469/469 [00:18<00:00, 25.26it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:18<00:00, 25.28it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:18<00:00, 25.19it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:18<00:00, 25.04it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:18<00:00, 25.09it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 118.64it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0448
	ExperienceForgetting/eval_phase/test_strea

 98%|█████████▊| 49/50 [3:27:31<03:11, 191.59s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1220
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.5442
	StreamForgetting/eval_phase/test_stream = 0.5442
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3541
-- >> Start of training phase << --
100%|██████████| 469/469 [00:18<00:00, 24.74it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:19<00:00, 24.64it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:18<00:00, 24.70it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:18<00:00, 24.75it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:19<00:00, 24.66it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 120.29it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0396
	ExperienceForgetting/eval_phase/test_strea

100%|██████████| 50/50 [3:29:44<00:00, 251.68s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.8210
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.5447
	StreamForgetting/eval_phase/test_stream = 0.5447
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3567


# Get metrics

In [12]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [13]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))